# Eyettention

In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import model
import torch
from torch.utils import model_zoo
import pandas as pd
from utils import *
from sklearn.model_selection import StratifiedKFold, KFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam, RMSprop
from transformers import BertTokenizerFast
from model import Eyettention
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torch.nn.functional import cross_entropy, softmax
from collections import deque, Counter
import pickle
import json
import matplotlib.pyplot as plt
import argparse
import random
from scasim import *
from evaluate_e_z_reader_model import *
from uniform_model import *
from evaluate_swift import *
import ast

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
save_data_folder = "./drive/MyDrive/results/celer/NRS"
#DEVICE = 'cuda'
DEVICE = 'cpu'
scanpath_gen_flag = True
atten_type = "local_g"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Training loop**

In [ ]:
if __name__ == '__main__':
	gpu = 0

	torch.set_default_tensor_type('torch.FloatTensor')
	availbl = torch.cuda.is_available()
	if availbl:
		device = f'cuda:{gpu}'
	else:
		device = 'cpu'
	#torch.cuda.set_device(gpu)

	cf = {"model_pretrained": "bert-base-cased",
			"lr": 1e-3,
			"max_grad_norm": 10,
			"n_epochs": 170,  # 1000
			"n_folds": 5,
			"dataset": 'celer',
			"atten_type": 'local-g',
			"batch_size": 256,
			"max_sn_len": 24, #include start token and end token
			"max_sn_token": 35,
			"max_sp_len": 52, #include start token and end token
			"max_sp_token": 395,
			"norm_type": "z-score",
			"earlystop_patience": 20,
			"max_pred_len": 60
			}

	#Encode the label into interger categories, setting the exclusive category 'cf["max_sn_len"]-1' as the end sign
	le = LabelEncoder()
	le.fit(np.append(np.arange(-cf["max_sn_len"]+3, cf["max_sn_len"]-1), cf["max_sn_len"]-1))
	#le.classes_

	#load corpus
	word_info_df, _, eyemovement_df = load_corpus(cf["dataset"])

	reader_list = celer_load_native_speaker()
	#Make list with sentence index
	sn_list = np.unique(word_info_df[word_info_df['list'].isin(reader_list)].sentenceid.values).tolist()

	sent_dict = convert_sent_id(sn_list)

	random.seed(0)
	fold_indx = 0
	#for scanpath generation
	sp_dnn_list = []
	sp_human_list = []
	for i in range(5):
		print('Sampling time:', i)
		loss_dict = {'val_loss':[], 'train_loss':[], 'test_ll':[], 'test_ll_SE':[], 'test_mse_dur':[], 'test_mse_dur_SE':[], 'test_mse_land_pos':[], 'test_mse_land_pos_SE':[], 'central_scasim_dnn':[], 'central_scasim_dnn_SE':[], 'central_scasim_human':[], 'central_scasim_human_SE':[], 'scasim_dnn':[], 'scasim_dnn_SE':[], 'scasim_human':[], 'scasim_human_SE':[], 'uniform_scasim':[], 'uniform_scasim_SE':[], 'uniform_central_scasim':[], 'uniform_central_scasim_SE':[], 'uniform_nll_SE':[], 'uniform_nll':[], 'uniform_mse_dur_SE':[], 'uniform_mse_dur':[], 'uniform_mse_land_pos_SE':[], 'uniform_mse_land_pos':[],
		             'ez_reader_scasim':[], 'ez_reader_scasim_SE':[], 'ez_reader_central_scasim':[], 'ez_reader_central_scasim_SE':[], 'ez_reader_mse_dur_SE':[], 'ez_reader_mse_dur':[], 'ez_reader_mse_land_pos_SE':[], 'ez_reader_mse_land_pos':[], 'SWIFT_scasim':[], 'SWIFT_scasim_SE':[], 'SWIFT_central_scasim':[], 'SWIFT_central_scasim_SE':[], 'SWIFT_nll_SE':[], 'SWIFT_nll':[], 'SWIFT_mse_dur_SE':[], 'SWIFT_mse_dur':[], 'SWIFT_mse_land_pos_SE':[], 'SWIFT_mse_land_pos':[]}
		readers_test = random.sample(reader_list, int(np.ceil(len(reader_list)*0.3)))
		reader_list_val = random.sample(readers_test, int(np.ceil(len(readers_test)*0.3)))
		reader_list_test = set(readers_test) - set(reader_list_val)
		reader_list_test = list(reader_list_test)
		reader_list_train = set(reader_list) - set(reader_list_test) - set(reader_list_val)
		reader_list_train = list(reader_list_train)
		#Make the order of the test samples unchanged on each re-run
		reader_list_test.sort()

		sn_test = random.sample(sn_list, int(np.ceil(len(sn_list)*0.3)))
		sn_list_val = random.sample(sn_test, int(np.ceil(len(sn_test)*0.3)))
		sn_list_test = set(sn_test) - set(sn_list_val)
		sn_list_test = list(sn_list_test)
		sn_list_train = set(sn_list) - set(sn_list_test) - set(sn_list_val)
		sn_list_train = list(sn_list_train)
		#Make the order of the test samples unchanged on each re-run
		sn_list_test.sort()

		#initialize tokenizer
		tokenizer = BertTokenizerFast.from_pretrained(cf['model_pretrained'])

		filtered_sent_dict_train = get_relevant_sent_dict(sent_dict, sn_list_train)
		filtered_sent_dict_val = get_relevant_sent_dict(sent_dict, sn_list_val)
		filtered_sent_dict_test = get_relevant_sent_dict(sent_dict, sn_list_test)

		#Preparing batch data
		dataset_train = celerdataset(word_info_df, eyemovement_df, cf, reader_list_train, sn_list_train, tokenizer, filtered_sent_dict_train)
		train_dataloaderr = DataLoader(dataset_train, batch_size = cf["batch_size"], shuffle = True, drop_last=True)

		dataset_val = celerdataset(word_info_df, eyemovement_df, cf, reader_list_val, sn_list_val, tokenizer, filtered_sent_dict_val)
		val_dataloaderr = DataLoader(dataset_val, batch_size = cf["batch_size"], shuffle = False, drop_last=True)

		dataset_test = celerdataset(word_info_df, eyemovement_df, cf, reader_list_test, sn_list_test, tokenizer, filtered_sent_dict_test)
		test_dataloaderr = DataLoader(dataset_test, batch_size = cf["batch_size"], shuffle = False, drop_last=False)

		#z-score normalization for gaze features
		fix_dur_mean, fix_dur_std = calculate_mean_std(dataloader=train_dataloaderr, feat_key="sp_fix_dur", padding_value=0, scale=1000)
		landing_pos_mean, landing_pos_std = calculate_mean_std(dataloader=train_dataloaderr, feat_key="sp_landing_pos", padding_value=0)
		sn_word_len_mean, sn_word_len_std = calculate_mean_std(dataloader=train_dataloaderr, feat_key="sn_word_len")

		# load model
		dnn = Eyettention(cf)
		#training
		episode = 0
		optimizer = Adam(dnn.parameters(), lr=cf["lr"])
		dnn.train()
		dnn.to(device)
		av_score = deque(maxlen=100)
		av_location_score = deque(maxlen=100)
		av_duration_score = deque(maxlen=100)
		av_land_pos_score = deque(maxlen=100)
		old_score = 1e10
		save_ep_couter = 0
		print('Start training')
		for episode_i in range(episode, cf["n_epochs"]+1):
			dnn.train()
			print('episode:', episode_i)
			counter = 0
			for batchh in train_dataloaderr:
				counter += 1
				batchh.keys()
				sn_ids = batchh["sn_ids"].to(device)
				sn_input_ids = batchh["sn_input_ids"].to(device)
				sn_attention_mask = batchh["sn_attention_mask"].to(device)
				word_ids_sn = batchh["word_ids_sn"].to(device)
				sn_word_len = batchh["sn_word_len"].to(device)

				sp_input_ids = batchh["sp_input_ids"].to(device)
				sp_attention_mask = batchh["sp_attention_mask"].to(device)
				word_ids_sp = batchh["word_ids_sp"].to(device)

				sp_pos = batchh["sp_pos"].to(device)
				sp_landing_pos = batchh["sp_landing_pos"].to(device) # [256, 40]
				sp_fix_dur = (batchh["sp_fix_dur"]/1000).to(device) # [256, 40]

				# normalize gaze features (z-score normalisation)
				mask = ~torch.eq(sp_fix_dur, 0)
				sp_fix_dur = (sp_fix_dur-fix_dur_mean)/fix_dur_std * mask
				sp_fix_dur = torch.nan_to_num(sp_fix_dur) # [256, 40]
				sp_landing_pos = (sp_landing_pos - landing_pos_mean)/landing_pos_std * mask
				sp_landing_pos = torch.nan_to_num(sp_landing_pos)
				sn_word_len = (sn_word_len - sn_word_len_mean)/sn_word_len_std
				sn_word_len = torch.nan_to_num(sn_word_len)
				sub_id = batchh["sub_id"].to(device)

				# zero old gradients
				optimizer.zero_grad()
				# predict output with DNN
				location_preds, duration_preds, landing_pos_preds, atten_weights = dnn(sn_emd=sn_input_ids,
											sn_mask=sn_attention_mask,
											sp_emd=sp_input_ids,
											sp_pos=sp_pos,
											word_ids_sn=word_ids_sn,
											word_ids_sp=word_ids_sp,
											sp_fix_dur=sp_fix_dur,
											sp_landing_pos=sp_landing_pos,
											sn_word_len = sn_word_len,
											sn_word_freq= None,
											sn_pred = None)#[batch, step, dec_o_dim]

				location_preds = location_preds.permute(0,2,1)              #[batch, dec_o_dim, step]

				#prepare label and mask
				# Compute loss for fixation locations
				pad_mask, label = load_label(sp_pos, cf, le, device)
				loss = nn.CrossEntropyLoss(reduction="none")
				batch_location_error = torch.mean(torch.masked_select(loss(location_preds, label), ~pad_mask))

				# Compute loss for fixation durations
				duration_labels = sp_fix_dur[:, :51] # Adjust duration_labels to match the sequence length of duration_preds
				duration_preds = duration_preds.squeeze(-1)  # Remove extra dimension (from [256, 39, 1] to [256, 39])
				dur_loss = nn.MSELoss(reduction="none")
				batch_duration_error = torch.mean(dur_loss(duration_preds, duration_labels))

				# Compute loss for landing position
				landing_pos_labels = sp_landing_pos[:, :51] # Adjust duration_labels to match the sequence length of duration_preds
				landing_pos_preds = landing_pos_preds.squeeze(-1)  # Remove extra dimension (from [256, 39, 1] to [256, 39])
				land_pos_loss = nn.MSELoss(reduction="none")
				batch_land_pos_error = torch.mean(land_pos_loss(landing_pos_preds, landing_pos_labels))

				# Combined loss for both location and duration
				batch_error = batch_location_error +  batch_duration_error + batch_land_pos_error

				# backpropagate loss
				batch_error.backward()
				# clip gradients
				gradient_clipping(dnn, cf["max_grad_norm"])

				#learn
				optimizer.step()
				av_location_score.append(batch_location_error.to('cpu').detach().numpy())
				av_duration_score.append(batch_duration_error.to('cpu').detach().numpy())
				av_land_pos_score.append(batch_land_pos_error.to('cpu').detach().numpy())
				av_score.append(batch_error.to('cpu').detach().numpy())
				print('counter:',counter)
				print('\rSample {}\tLocation Loss: {:.10f}\tDuration Loss: {:.10f}\tLanding position Loss: {:.10f}'.format(
          	counter, np.mean(av_location_score), np.mean(av_duration_score), np.mean(av_land_pos_score)), end=" ")
			loss_dict['train_loss'].append(np.mean(av_score))
			if np.mean(av_score) < old_score:
				# save model if val loss is smallest
				torch.save(dnn.state_dict(), '{}/CELER_3head_arch_NRS_{}.pth'.format(save_data_folder, fold_indx))
				old_score = np.mean(av_score)
				print('\nsaved model state dict\n')
				save_ep_couter = episode_i
			else:
				#early stopping
				if episode_i - save_ep_couter >= cf["earlystop_patience"]:
					break

			location_val_loss = []
			duration_val_loss = []
			land_pos_val_loss = []
			val_loss = []
			dnn.eval()
			for batchh in val_dataloaderr:
				with torch.no_grad():
					sn_ids_val = batchh["sn_ids"].to(device)
					sn_input_ids_val = batchh["sn_input_ids"].to(device)
					sn_attention_mask_val = batchh["sn_attention_mask"].to(device)
					word_ids_sn_val = batchh["word_ids_sn"].to(device)
					sn_word_len_val = batchh["sn_word_len"].to(device)

					sp_input_ids_val = batchh["sp_input_ids"].to(device)
					sp_attention_mask_val = batchh["sp_attention_mask"].to(device)
					word_ids_sp_val = batchh["word_ids_sp"].to(device)

					sp_pos_val = batchh["sp_pos"].to(device)
					sp_landing_pos_val = batchh["sp_landing_pos"].to(device)
					sp_fix_dur_val = (batchh["sp_fix_dur"]/1000).to(device)

					#normalize gaze features
					mask = ~torch.eq(sp_fix_dur_val, 0)
					sp_fix_dur_val = (sp_fix_dur_val-fix_dur_mean)/fix_dur_std * mask
					sp_landing_pos_val = (sp_landing_pos_val - landing_pos_mean)/landing_pos_std * mask
					sp_fix_dur_val = torch.nan_to_num(sp_fix_dur_val)
					sp_landing_pos_val = torch.nan_to_num(sp_landing_pos_val)
					sn_word_len_val = (sn_word_len_val - sn_word_len_mean)/sn_word_len_std
					sn_word_len_val = torch.nan_to_num(sn_word_len_val)
					sub_id_val = batchh["sub_id"].to(device)

					location_preds_val, duration_preds_val, landing_pos_preds_val, atten_weights_val = dnn(sn_emd=sn_input_ids_val,
														sn_mask=sn_attention_mask_val,
														sp_emd=sp_input_ids_val,
														sp_pos=sp_pos_val,
														word_ids_sn=word_ids_sn_val,
														word_ids_sp=word_ids_sp_val,
														sp_fix_dur=sp_fix_dur_val,
														sp_landing_pos=sp_landing_pos_val,
														sn_word_len = sn_word_len_val,
														sn_word_freq= None,
											      sn_pred = None)#[batch, step, dec_o_dim]
					location_preds_val = location_preds_val.permute(0,2,1)              #[batch, dec_o_dim, step

					# Compute location prediction error
					loss = nn.CrossEntropyLoss(reduction="none")
					pad_mask_val, label_val = load_label(sp_pos_val, cf, le, device)
					location_error_val = torch.mean(torch.masked_select(loss(location_preds_val, label_val), ~pad_mask_val))
					location_val_loss.append(location_error_val.detach().to('cpu').numpy())

					# Compute duration prediction error
					duration_labels_val = sp_fix_dur_val[:, :51] # Adjust duration_labels to match the sequence length of duration_preds
					duration_preds_val = duration_preds_val.squeeze(-1)
					dur_loss = nn.MSELoss(reduction="none")
					duration_error_val = torch.mean(dur_loss(duration_preds_val, duration_labels_val))
					duration_val_loss.append(duration_error_val.detach().to('cpu').numpy())

					# Compute loss for landing position
					landing_pos_labels_val = sp_landing_pos_val[:, :51] # Adjust duration_labels to match the sequence length of duration_preds
					landing_pos_preds_val = landing_pos_preds_val.squeeze(-1)  # Remove extra dimension (from [256, 39, 1] to [256, 39])
					land_pos_error_val = torch.mean(land_pos_loss(landing_pos_preds_val, landing_pos_labels_val))
					land_pos_val_loss.append(land_pos_error_val.detach().to('cpu').numpy())

					combined_loss = location_error_val + duration_error_val + land_pos_error_val
					val_loss.append(combined_loss.detach().to('cpu').numpy())

			print('\nValidation loss for locations {} \n'.format(np.mean(location_val_loss)))
			print('\nValidation loss for duration {} \n'.format(np.mean(duration_val_loss)))
			print('\nValidation loss for landing position {} \n'.format(np.mean(land_pos_val_loss)))
			loss_dict['val_loss'].append(np.mean(val_loss))

			if np.mean(val_loss) < old_score:
				# save model if val loss is smallest
				torch.save(dnn.state_dict(), '{}/CELER_3head_arch_NRS_{}.pth'.format(save_data_folder, fold_indx))
				old_score = np.mean(val_loss)
				print('\nsaved model state dict\n')
				save_ep_couter = episode_i
			else:
				#early stopping
				if episode_i - save_ep_couter >= cf["earlystop_patience"]:
					break
		fold_indx += 1

		#evaluation
		dnn.eval()
		res_llh=[]
		res_mse_dur = []
		res_mse_land_pos = []
		res_central_scasim_human = []
		res_central_scasim_dnn = []
		res_scasim_human = []
		res_scasim_dnn = []
		uniform_central_scasim_scores = []
		uniform_scasim_scores = []
		uniform_nll_scores = []
		uniform_mse_dur_scores = []
		uniform_mse_land_pos_scores = []
		ez_reader_central_scasim_scores = []
		ez_reader_scasim_scores = []
		ez_reader_nll_scores = []
		ez_reader_mse_dur_scores = []
		ez_reader_mse_land_pos_scores = []
		swift_central_scasim_scores = []
		swift_scasim_scores = []
		swift_nll_scores = []
		swift_mse_dur_scores = []
		swift_mse_land_pos_scores = []
		dnn.load_state_dict(torch.load(os.path.join(save_data_folder, f'CELER_3head_arch_NRS_{fold_indx}.pth'), map_location='cpu'))
		dnn.to(device)
		batch_indx = 0
		for batchh in test_dataloaderr:
			with torch.no_grad():
				sn_ids_test = batchh["sn_ids"].to(device)
				sn_input_ids_test = batchh["sn_input_ids"].to(device)
				sn_attention_mask_test = batchh["sn_attention_mask"].to(device)
				word_ids_sn_test = batchh["word_ids_sn"].to(device)
				sn_word_len_test = batchh["sn_word_len"].to(device)

				sp_input_ids_test = batchh["sp_input_ids"].to(device)
				sp_attention_mask_test = batchh["sp_attention_mask"].to(device)
				word_ids_sp_test = batchh["word_ids_sp"].to(device)

				sp_pos_test = batchh["sp_pos"].to(device) # 28: '<Sep>', 29: '<'Pad'>'
				sp_landing_pos_test = batchh["sp_landing_pos"].to(device)
				sp_fix_dur_test = (batchh["sp_fix_dur"]/1000).to(device)

				#normalize gaze features
				mask = ~torch.eq(sp_fix_dur_test, 0)
				sp_fix_dur_test = (sp_fix_dur_test-fix_dur_mean)/fix_dur_std * mask
				sp_landing_pos_test = (sp_landing_pos_test - landing_pos_mean)/landing_pos_std * mask
				sp_fix_dur_test = torch.nan_to_num(sp_fix_dur_test)
				sp_landing_pos_test = torch.nan_to_num(sp_landing_pos_test)
				sn_word_len_test = (sn_word_len_test - sn_word_len_mean)/sn_word_len_std
				sn_word_len_test = torch.nan_to_num(sn_word_len_test)

				location_preds_test, duration_preds_test, landing_pos_preds_test, atten_weights_test = dnn(sn_emd=sn_input_ids_test,
														sn_mask=sn_attention_mask_test,
														sp_emd=sp_input_ids_test,
														sp_pos=sp_pos_test,
														word_ids_sn=word_ids_sn_test,
														word_ids_sp=word_ids_sp_test,
														sp_fix_dur=sp_fix_dur_test,
														sp_landing_pos=sp_landing_pos_test,
														sn_word_len = sn_word_len_test,
														sn_pred=None,
                            sn_word_freq=None) #[batch, step, dec_o_dim]


				########## Evaluate location predictions ##########
				m = nn.Softmax(dim=2)
				location_preds_test = m(location_preds_test).detach().to('cpu').numpy()
				#prepare label and mask
				pad_mask_test, label_test = load_label(sp_pos_test, cf, le, 'cpu')
				#compute log likelihood for the batch samples
				res_batch = eval_log_llh(location_preds_test, label_test, pad_mask_test)
				res_llh.append(np.array(res_batch))

				uniform_output = construct_uniform_tensor(location_preds_test)
				uniform_nll = eval_log_llh(uniform_output, label_test, pad_mask_test)
				uniform_nll_scores.append(np.array(uniform_nll))
				print("Uniform_nll", np.mean(uniform_nll), uniform_nll)

				duration_preds_test = duration_preds_test.squeeze(-1)
				duration_labels_test = sp_fix_dur_test[:, :51]
				test_mask = mask[:, :51]
				mse_dur = eval_mse(duration_preds_test, duration_labels_test, test_mask)
				print("MSE for durations", np.mean(mse_dur))
				res_mse_dur.append(np.array(mse_dur))

				landing_pos_preds_test = landing_pos_preds_test.squeeze(-1)
				landing_pos_labels_test = sp_landing_pos_test[:, :51]
				mse_landing_pos = eval_mse(landing_pos_preds_test, landing_pos_labels_test, test_mask)
				print("MSE for landing positions", np.mean(mse_landing_pos))
				res_mse_land_pos.append(np.array(mse_landing_pos))

				if bool(scanpath_gen_flag) == True:
					sn_len = (torch.max(torch.nan_to_num(word_ids_sn_test), dim=1)[0]+1-2).detach().to('cpu').numpy()
					# compute the scan path generated from the model when the first CLS token is given
					sp_dnn, _, dur_dnn, land_pos_dnn = dnn.scanpath_generation(sn_emd=sn_input_ids_test,
														 sn_mask=sn_attention_mask_test,
														 word_ids_sn=word_ids_sn_test,
														 sn_word_len = sn_word_len_test,
														 le=le,
														 sn_word_freq=None,
                             sn_pred=None,
                             sp_fix_dur=sp_fix_dur_test,
														 sp_landing_pos = sp_landing_pos_test,
														 max_pred_len=cf['max_pred_len'])


					sp_dnn, sp_human = prepare_scanpath(sp_dnn.detach().to('cpu').numpy(),
                                              dur_dnn.detach().to('cpu').numpy(),
																							land_pos_dnn.detach().to('cpu').numpy(), sn_len, sp_pos_test,
                                              sp_fix_dur_test, sp_landing_pos_test, cf, sn_ids_test, fix_dur_mean, fix_dur_std, landing_pos_mean, landing_pos_std)

					sp_dnn_list.extend(sp_dnn)
					sp_human_list.extend(sp_human)

					sp_dnn = filter_sp(sp_dnn)
					sp_human = filter_sp(sp_human)

					if len(sp_dnn["sent_id"]) != 0:
						# evaluate only on sent that were read by multiple subjects
						sp_dnn = convert_sp_to_lists(sp_dnn)
						sp_human = convert_sp_to_lists(sp_human)
						sp_human = modify_landing_pos(sp_human.copy())
						sp_dnn = modify_landing_pos(sp_dnn.copy())
						random_sp = sample_random_sp("CELER", sp_human, 'data_splits/CELER_sent_dict.txt')
						random_sp = convert_sp_to_lists(random_sp)
						random_sp = modify_landing_pos(random_sp.copy())


						print("######### Eyettention model evaluation ##########")
						scasim_scores_dnn = compute_scasim(sp_dnn, sp_human)
						res_scasim_dnn.append(scasim_scores_dnn)
						print("Mean scasim dnn", np.mean(scasim_scores_dnn))
						scasim_scores_human = compute_scasim(sp_human, random_sp)
						res_scasim_human.append(scasim_scores_human)
						print("Mean scasim human", np.mean(scasim_scores_human))

						central_scasim_scores_dnn = compute_central_scasim("CELER_most_central_sp.txt", sp_dnn, 'data_splits/CELER_sent_dict.txt')
						central_scasim_scores_human = compute_central_scasim("CELER_most_central_sp.txt", sp_human, 'data_splits/CELER_sent_dict.txt')
						res_central_scasim_dnn.append(np.array(central_scasim_scores_dnn))
						res_central_scasim_human.append(np.array(central_scasim_scores_human))
						print("Mean central scasim dnn", np.mean(central_scasim_scores_dnn))
						print("Mean central scasim human", np.mean(central_scasim_scores_human))

						print("######### Uniform baseline model evaluation ##########")
						mean_dur_uniform, std_dur_uniform, mean_land_pos_uniform, std_land_pos_uniform = compute_mean_std_uniform("drive/MyDrive/baseline/Uniform/CELER_uniform_results.csv")
						uniform_central_scasim, uniform_scasim, dur_mse_scores, land_pos_mse_scores = evaluate_uniform_model("CELER", sp_human, landing_pos_mean, landing_pos_std, fix_dur_mean, fix_dur_std, mean_land_pos_uniform, std_land_pos_uniform, mean_dur_uniform, std_dur_uniform)
						uniform_central_scasim_scores.append(np.array(uniform_central_scasim))
						uniform_scasim_scores.append(np.array(uniform_scasim))
						uniform_mse_dur_scores.append(np.array(dur_mse_scores))
						uniform_mse_land_pos_scores.append(np.array(land_pos_mse_scores))
						print("Uniform mean central Scasim score:", np.mean(uniform_central_scasim))
						print("Uniform mean Scasim score:", np.mean(uniform_scasim))
						print("MSE for uniform durations", np.mean(dur_mse_scores))
						print("MSE for uniform landing pos", np.mean(land_pos_mse_scores))

						print("######### E-Z Reader model evaluation ##########")
						mean_dur_ez_reader, std_dur_ez_reader, mean_land_pos_ez_reader, std_land_pos_ez_reader = compute_mean_std_ez_reader("drive/MyDrive/baseline/CELERSimulationResultsNew.txt")
						print(mean_dur_ez_reader, std_dur_ez_reader, mean_land_pos_ez_reader, std_land_pos_ez_reader)
						central_scasim_ez_reader, scasim_ez_reader, dur_mse_ez_reader, land_pos_mse_ez_reader = evaluate_ez_reader("CELER", "drive/MyDrive/baseline/CELERSimulationResultsNew.txt", sp_human, landing_pos_mean, landing_pos_std, fix_dur_mean, fix_dur_std, mean_land_pos_ez_reader, std_land_pos_ez_reader, mean_dur_ez_reader, std_dur_ez_reader)
						ez_reader_central_scasim_scores.append(np.array(central_scasim_ez_reader))
						print("Central scasim E-Z Reader", np.mean(central_scasim_ez_reader))
						ez_reader_scasim_scores.append(np.array(scasim_ez_reader))
						print("Mean scasim score E-Z Reader", np.mean(scasim_ez_reader))
						ez_reader_mse_dur_scores.append(np.array(dur_mse_ez_reader))
						print("MSE for durations E-Z Reader", np.mean(dur_mse_ez_reader))
						ez_reader_mse_land_pos_scores.append(np.array(land_pos_mse_ez_reader))
						print("MSE for landing pos E-Z Reader", np.mean(land_pos_mse_ez_reader))

						print("######### SWIFT model evaluation ##########")
						mean_dur_swift, std_dur_swift, mean_land_pos_swift, std_land_pos_swift = compute_mean_std_swift()
						central_scasim_swift, scasim_swift, dur_mse_swift, land_pos_mse_swift = evaluate_swift("celer", sp_human, landing_pos_mean, landing_pos_std, fix_dur_mean, fix_dur_std, mean_land_pos_swift, std_land_pos_swift, mean_dur_swift, std_dur_swift)
						swift_central_scasim_scores.append(np.array(central_scasim_swift))
						print("Central scasim SWIFT", np.mean(central_scasim_swift))
						swift_scasim_scores.append(np.array(scasim_swift))
						print("Mean scasim score SWIFT", np.mean(scasim_swift))
						swift_mse_dur_scores.append(np.array(dur_mse_swift))
						print("MSE for durations SWIFT", np.mean(dur_mse_swift))
						swift_mse_land_pos_scores.append(np.array(land_pos_mse_swift))
						print("MSE for landing pos SWIFT", np.mean(land_pos_mse_swift))
						swift_nll = evaluate_swift_nll("baseline/swift/SWIFT_NLL.xlsx", sp_human)
						swift_nll_scores.append(np.array(swift_nll))
						if len(swift_nll) != 0:
							print("NLL SWIFT", np.mean(swift_nll))

				batch_indx +=1

		res_llh = np.concatenate(res_llh).ravel()
		loss_dict['test_ll'].append(res_llh)
		res_mse_dur = np.concatenate(res_mse_dur).ravel()
		loss_dict['test_mse_dur'].append(res_mse_dur)
		res_mse_land_pos = np.concatenate(res_mse_land_pos).ravel()
		loss_dict['test_mse_land_pos'].append(res_mse_land_pos)

		res_central_scasim_dnn = np.concatenate(res_central_scasim_dnn).ravel()
		loss_dict['central_scasim_dnn'].append(res_central_scasim_dnn)
		res_central_scasim_human = np.concatenate(res_central_scasim_human).ravel()
		loss_dict['central_scasim_human'].append(res_central_scasim_human)
		res_scasim_dnn = np.concatenate(res_scasim_dnn).ravel()
		loss_dict['scasim_dnn'].append(res_scasim_dnn)
		res_scasim_human = np.concatenate(res_scasim_human).ravel()
		loss_dict['scasim_human'].append(res_scasim_human)

		loss_dict['fix_dur_mean'] = fix_dur_mean
		loss_dict['fix_dur_std'] = fix_dur_std
		loss_dict['landing_pos_mean'] = landing_pos_mean
		loss_dict['landing_pos_std'] = landing_pos_std
		loss_dict['sn_word_len_mean'] = sn_word_len_mean
		loss_dict['sn_word_len_std'] = sn_word_len_std

		uniform_central_scasim_scores = np.concatenate(uniform_central_scasim_scores).ravel()
		loss_dict['uniform_central_scasim'].append(uniform_central_scasim_scores)
		uniform_scasim_scores = np.concatenate(uniform_scasim_scores).ravel()
		loss_dict['uniform_scasim'].append(uniform_scasim_scores)

		uniform_mse_dur_scores = np.concatenate(uniform_mse_dur_scores).ravel()
		loss_dict['uniform_mse_dur'].append(uniform_mse_dur_scores)
		uniform_mse_land_pos_scores = np.concatenate(uniform_mse_land_pos_scores).ravel()
		loss_dict['uniform_mse_land_pos'].append(uniform_mse_land_pos_scores)
		uniform_nll_scores = np.concatenate(uniform_nll_scores).ravel()
		loss_dict['uniform_nll'].append(uniform_nll_scores)

		ez_reader_central_scasim_scores = np.concatenate(ez_reader_central_scasim_scores).ravel()
		loss_dict['ez_reader_central_scasim'].append(ez_reader_central_scasim_scores)
		ez_reader_scasim_scores = np.concatenate(ez_reader_scasim_scores).ravel()
		loss_dict['ez_reader_scasim'].append(ez_reader_scasim_scores)

		ez_reader_mse_dur_scores = np.concatenate(ez_reader_mse_dur_scores).ravel()
		loss_dict['ez_reader_mse_dur'].append(ez_reader_mse_dur_scores)
		ez_reader_mse_land_pos_scores = np.concatenate(ez_reader_mse_land_pos_scores).ravel()
		loss_dict['ez_reader_mse_land_pos'].append(ez_reader_mse_land_pos_scores)

		swift_mse_dur_scores = np.concatenate(swift_mse_dur_scores).ravel()
		loss_dict['SWIFT_mse_dur'].append(swift_mse_dur_scores)
		swift_mse_land_pos_scores = np.concatenate(swift_mse_land_pos_scores).ravel()
		loss_dict['SWIFT_mse_land_pos'].append(swift_mse_land_pos_scores)
		swift_nll_scores = np.concatenate(swift_nll_scores).ravel()
		loss_dict['SWIFT_nll'].append(swift_nll_scores)
		swift_central_scasim_scores = np.concatenate(swift_central_scasim_scores).ravel()
		loss_dict['SWIFT_central_scasim'].append(swift_central_scasim_scores)
		swift_scasim_scores = np.concatenate(swift_scasim_scores).ravel()
		loss_dict['SWIFT_scasim'].append(swift_scasim_scores)

		print('Test likelihood is {}'.format(np.mean(res_llh)))
		loss_dict['test_ll_SE'].append(np.std(res_llh)/ np.sqrt(len(res_llh)))
		print("Standard error for NLL", np.std(res_llh)/ np.sqrt(len(res_llh)))

		print('Test MSE for durations is {}'.format(np.mean(res_mse_dur)))
		loss_dict['test_mse_dur_SE'].append(np.std(res_mse_dur)/ np.sqrt(len(res_mse_dur)))
		print("Standard error for MSE dur", np.std(res_mse_dur) / np.sqrt(len(res_mse_dur)))

		print('Test MSE for landing positions is {}'.format(np.mean(res_mse_land_pos)))
		loss_dict['test_mse_land_pos_SE'].append(np.std(res_mse_land_pos)/ np.sqrt(len(res_mse_land_pos)))
		print("Standard error for MSE land pos", np.std(res_mse_land_pos) / np.sqrt(len(res_mse_land_pos)))

		print("Central Scasim dnn", np.mean(loss_dict['central_scasim_dnn']))
		loss_dict['central_scasim_dnn_SE'].append(np.std(res_central_scasim_dnn)/ np.sqrt(len(res_central_scasim_dnn)))
		print("Standard error for Central scasim DNN", np.std(res_central_scasim_dnn) / np.sqrt(len(res_central_scasim_dnn)))

		print("Central Scasim human", np.mean(loss_dict['central_scasim_human']))
		loss_dict['central_scasim_human_SE'].append(np.std(res_central_scasim_human)/ np.sqrt(len(res_central_scasim_human)))
		print("Standard error for Central scasim human", np.std(res_central_scasim_human) / np.sqrt(len(res_central_scasim_human)))

		print("Scasim dnn", np.mean(loss_dict['scasim_dnn']))
		loss_dict['scasim_dnn_SE'].append(np.std(res_scasim_dnn)/ np.sqrt(len(res_scasim_dnn)))
		print("Standard error for scasim dnn", np.std(res_scasim_dnn) / np.sqrt(len(res_scasim_dnn)))

		print("Scasim human", np.mean(loss_dict['scasim_human']))
		loss_dict['scasim_human_SE'].append(np.std(res_scasim_human)/ np.sqrt(len(res_scasim_human)))
		print("Standard error for scasim human", np.std(res_scasim_human) / np.sqrt(len(res_scasim_human)))

		print("Uniform central scasim", np.mean(loss_dict['uniform_central_scasim']))
		loss_dict['uniform_central_scasim_SE'].append(np.std(uniform_central_scasim_scores)/ np.sqrt(len(uniform_central_scasim_scores)))
		print("Standard error for uniform central scasim", np.std(uniform_central_scasim_scores) / np.sqrt(len(uniform_central_scasim_scores)))

		print("Uniform scasim", np.mean(loss_dict['uniform_scasim']))
		loss_dict['uniform_scasim_SE'].append(np.std(uniform_scasim_scores)/ np.sqrt(len(uniform_scasim_scores)))
		print("Standard error for uniform scasim", np.std(uniform_scasim_scores) / np.sqrt(len(uniform_scasim_scores)))

		print("Uniform MSE durations", np.mean(loss_dict['uniform_mse_dur']))
		loss_dict['uniform_mse_dur_SE'].append(np.std(dur_mse_scores)/ np.sqrt(len(dur_mse_scores)))
		print("Standard error for uniform MSE durations", np.std(dur_mse_scores) / np.sqrt(len(dur_mse_scores)))

		print("Uniform MSE landing pos", np.mean(loss_dict['uniform_mse_land_pos']))
		loss_dict['uniform_mse_land_pos_SE'].append(np.std(land_pos_mse_scores)/ np.sqrt(len(land_pos_mse_scores)))
		print("Standard error for uniform MSE landing pos", np.std(land_pos_mse_scores) / np.sqrt(len(land_pos_mse_scores)))

		print("Uniform NLL", np.mean(loss_dict['uniform_nll']))
		loss_dict['uniform_nll_SE'].append(np.std(uniform_nll_scores)/ np.sqrt(len(uniform_nll_scores)))
		print("Standard error for uniform NLL", np.std(uniform_nll_scores) / np.sqrt(len(uniform_nll_scores)))

		print("E-Z Reader central scasim", np.mean(loss_dict['ez_reader_central_scasim']))
		loss_dict['ez_reader_central_scasim_SE'].append(np.std(ez_reader_central_scasim_scores)/ np.sqrt(len(ez_reader_central_scasim_scores)))
		print("Standard error for E-Z Reader central scasim", np.std(ez_reader_central_scasim_scores) / np.sqrt(len(ez_reader_central_scasim_scores)))

		print("E-Z Reader scasim", np.mean(loss_dict['ez_reader_scasim']))
		loss_dict['ez_reader_scasim_SE'].append(np.std(ez_reader_scasim_scores)/ np.sqrt(len(ez_reader_scasim_scores)))
		print("Standard error for E-Z Reader scasim", np.std(ez_reader_scasim_scores) / np.sqrt(len(ez_reader_scasim_scores)))

		print("E-Z Reader MSE durations", np.mean(loss_dict['ez_reader_mse_dur']))
		loss_dict['ez_reader_mse_dur_SE'].append(np.std(ez_reader_mse_dur_scores)/ np.sqrt(len(ez_reader_mse_dur_scores)))
		print("Standard error for E-Z Reader MSE durations", np.std(ez_reader_mse_dur_scores) / np.sqrt(len(ez_reader_mse_dur_scores)))

		print("E-Z Reader MSE landing pos", np.mean(loss_dict['ez_reader_mse_land_pos']))
		loss_dict['ez_reader_mse_land_pos_SE'].append(np.std(ez_reader_mse_land_pos_scores)/ np.sqrt(len(ez_reader_mse_land_pos_scores)))
		print("Standard error for E-Z Reader MSE landing pos", np.std(ez_reader_mse_land_pos_scores) / np.sqrt(len(ez_reader_mse_land_pos_scores)))

		print("SWIFT NLL", np.mean(loss_dict['SWIFT_nll']))
		loss_dict['SWIFT_nll_SE'].append(np.std(swift_nll_scores)/ np.sqrt(len(swift_nll_scores)))
		print("Standard error for SWIFT NLL", np.std(swift_nll_scores) / np.sqrt(len(swift_nll_scores)))

		print("SWIFT central scasim", np.mean(loss_dict['SWIFT_central_scasim']))
		loss_dict['SWIFT_central_scasim_SE'].append(np.std(swift_central_scasim_scores)/ np.sqrt(len(swift_central_scasim_scores)))
		print("Standard error for SWIFT central scasim", np.std(swift_central_scasim_scores) / np.sqrt(len(swift_central_scasim_scores)))

		print("SWIFT scasim", np.mean(loss_dict['SWIFT_scasim']))
		loss_dict['SWIFT_scasim_SE'].append(np.std(swift_scasim_scores)/ np.sqrt(len(swift_scasim_scores)))
		print("Standard error for SWIFT scasim", np.std(swift_scasim_scores) / np.sqrt(len(swift_scasim_scores)))

		print("SWIFT MSE durations", np.mean(loss_dict['SWIFT_mse_dur']))
		loss_dict['SWIFT_mse_dur_SE'].append(np.std(swift_mse_dur_scores)/ np.sqrt(len(swift_mse_dur_scores)))
		print("Standard error for SWIFT MSE durations", np.std(swift_mse_dur_scores) / np.sqrt(len(swift_mse_dur_scores)))

		print("SWIFT MSE landing pos", np.mean(loss_dict['SWIFT_mse_land_pos']))
		loss_dict['SWIFT_mse_land_pos_SE'].append(np.std(swift_mse_land_pos_scores)/ np.sqrt(len(swift_mse_land_pos_scores)))
		print("Standard error for SWIFT MSE landing pos", np.std(swift_mse_land_pos_scores) / np.sqrt(len(swift_mse_land_pos_scores)))

		#save results
		with open('{}/res_CELER_eyettention_Fold{}.pickle'.format(save_data_folder, fold_indx), 'wb') as handle:
			pickle.dump(loss_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
		fold_indx += 1


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)
/content/utils.py:38: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  eyemovement_df = pd.read_csv('drive/MyDrive/celer/sent_fix.tsv', delimiter='\t')


Sampling time: 0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

keeping Bert with pre-trained weights
Uniform_nll -5.4918532371521 [-5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.49185323715

/content/utils.py:307: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)
/content/scasim.py:101: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  eyemovement_df = pd.read_csv('drive/MyDrive/celer/sent_fix.tsv', delimiter='\t')  # TODO ././Data/celer/


######### Eyettention model evaluation ##########
Mean scasim dnn 2891.246575342466
Mean scasim human 2447.2876712328766
Mean central scasim dnn 2296.582191780822
Mean central scasim human 2017.2671232876712
######### SWIFT model evaluation ##########
Central scasim SWIFT 1893.0076335877864
Mean scasim score SWIFT 3017.5496183206105
MSE for durations SWIFT 1.6956205019969066
MSE for landing pos SWIFT 1.6361783852104013
NLL SWIFT -5211278.904109589
Uniform_nll -5.4918532371521 [-5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.4918532371521, -5.49185323715

/content/utils.py:307: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)
/content/scasim.py:101: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  eyemovement_df = pd.read_csv('drive/MyDrive/celer/sent_fix.tsv', delimiter='\t')  # TODO ././Data/celer/


######### Eyettention model evaluation ##########
Mean scasim dnn 5802.555555555556
Mean scasim human 4813.7555555555555
Mean central scasim dnn 4779.111111111111
Mean central scasim human 3160.8444444444444
######### SWIFT model evaluation ##########
Central scasim SWIFT 2801.0232558139537
Mean scasim score SWIFT 4640.488372093023
MSE for durations SWIFT 1.8029148474682208
MSE for landing pos SWIFT 1.6150491736656012
Requested count (12) exceeds available scanpaths (0) for sentence 2557. Using all available scanpaths.
NLL SWIFT -5502621.575757576
SWIFT NLL -5264990.122905028
Standard error for SWIFT NLL 43655.551843790825
SWIFT central scasim 2117.402298850575
Standard error for SWIFT central scasim 91.70878069791019
SWIFT scasim 3418.6206896551726
Standard error for SWIFT scasim 152.8979808264598
SWIFT MSE durations 1.7221357712800476
Standard error for SWIFT MSE durations 0.14528838623226825
SWIFT MSE landing pos 1.6309567984493298
Standard error for SWIFT MSE landing pos 0.01201849

In [ ]:
import pickle
import numpy as np

# Analysing the results
test_ll_all_folds = []
test_mse_dur_all_folds = []
test_mse_land_pos_all_folds = []
scasim_dnn_all_folds = []
scasim_human_all_folds = []
central_scasim_dnn_all_folds = []
central_scasim_human_all_folds = []

uniform_ll_all_folds = []
uniform_mse_dur_all_folds = []
uniform_mse_land_pos_all_folds = []
uniform_scasim_all_folds = []
uniform_central_scasim_all_folds = []

ez_reader_mse_dur_all_folds = []
ez_reader_mse_land_pos_all_folds = []
ez_reader_scasim_all_folds = []
ez_reader_central_scasim_all_folds = []

fold_index = 0
for i in range(5):
  with open(f'drive/MyDrive/results/celer/NRS/res_CELER_eyettention_Fold{fold_index}.pickle', 'rb') as original:
      reader_results = pickle.load(original)
      test_ll_all_folds.extend(reader_results['test_ll'][0])
      test_mse_dur_all_folds.extend(reader_results['test_mse_dur'][0])
      test_mse_land_pos_all_folds.extend(reader_results['test_mse_land_pos'][0])
      scasim_dnn_all_folds.extend(reader_results['scasim_dnn'][0])
      scasim_human_all_folds.extend(reader_results['scasim_human'][0])
      central_scasim_dnn_all_folds.extend(reader_results['central_scasim_dnn'][0])
      central_scasim_human_all_folds.extend(reader_results['central_scasim_human'][0])

      uniform_ll_all_folds.extend(reader_results['uniform_nll'][0])
      uniform_mse_dur_all_folds.extend(reader_results['uniform_mse_dur'][0])
      uniform_mse_land_pos_all_folds.extend(reader_results['uniform_mse_land_pos'][0])
      uniform_scasim_all_folds.extend(reader_results['uniform_scasim'][0])
      uniform_central_scasim_all_folds.extend(reader_results['uniform_central_scasim'][0])

      ez_reader_mse_dur_all_folds.extend(reader_results['ez_reader_mse_dur'][0])
      ez_reader_mse_land_pos_all_folds.extend(reader_results['ez_reader_mse_land_pos'][0])
      ez_reader_scasim_all_folds.extend(reader_results['ez_reader_scasim'][0])
      ez_reader_central_scasim_all_folds.extend(reader_results['ez_reader_central_scasim'][0])

      fold_index += 1

# Calculate overall statistics
def calculate_mean_and_se(values):
    mean_val = np.mean(values)
    se_val = np.std(values) / np.sqrt(len(values))
    return mean_val, se_val

mean_nll, se_nll = calculate_mean_and_se(test_ll_all_folds)
mean_mse_dur, se_mse_dur = calculate_mean_and_se(test_mse_dur_all_folds)
mean_mse_land_pos, se_mse_land_pos = calculate_mean_and_se(test_mse_land_pos_all_folds)
mean_scasim, se_scasim = calculate_mean_and_se(scasim_dnn_all_folds)
mean_central_scasim, se_central_scasim = calculate_mean_and_se(central_scasim_dnn_all_folds)
mean_scasim_human, se_scasim_human = calculate_mean_and_se(scasim_human_all_folds)
mean_central_scasim_human, se_central_scasim_human = calculate_mean_and_se(central_scasim_human_all_folds)

mean_nll_uniform, se_nll_uniform = calculate_mean_and_se(uniform_ll_all_folds)
mean_mse_dur_uniform, se_mse_dur_uniform = calculate_mean_and_se(uniform_mse_dur_all_folds)
mean_mse_land_pos_uniform, se_mse_land_pos_uniform = calculate_mean_and_se(uniform_mse_land_pos_all_folds)
mean_scasim_uniform, se_scasim_uniform = calculate_mean_and_se(uniform_scasim_all_folds)
mean_central_scasim_uniform, se_central_scasim_uniform = calculate_mean_and_se(uniform_central_scasim_all_folds)

mean_nll_ez_reader, se_nll_ez_reader = calculate_mean_and_se(ez_reader_mse_dur_all_folds)
mean_mse_dur_ez_reader, se_mse_dur_ez_reader = calculate_mean_and_se(ez_reader_mse_dur_all_folds)
mean_mse_land_pos_ez_reader, se_mse_land_pos_ez_reader = calculate_mean_and_se(ez_reader_mse_land_pos_all_folds)
mean_scasim_ez_reader, se_scasim_ez_reader = calculate_mean_and_se(ez_reader_scasim_all_folds)
mean_central_scasim_ez_reader, se_central_scasim_ez_reader = calculate_mean_and_se(ez_reader_central_scasim_all_folds)


print("Overall Mean NLL:", mean_nll)
print("Overall Standard error NLL:", se_nll)

print("Overall Mean MSE dur:", mean_mse_dur)
print("Overall Standard error MSE dur:", se_mse_dur)

print("Overall Mean MSE landing pos:", mean_mse_land_pos)
print("Overall Standard error MSE landing pos:", se_mse_land_pos)

print("Overall Mean scasim:", mean_scasim)
print("Overall Standard error scasim:", se_scasim)

print("Overall Mean Central scasim:", mean_central_scasim)
print("Overall Standard error Central scasim:", se_central_scasim)

print("Human Mean scasim:", mean_scasim_human)
print("Human Standard error scasim:", se_scasim_human)

print("Human Mean central scasim:", mean_central_scasim_human)
print("Human Standard error central scasim:", se_central_scasim_human)


print("Overall Mean NLL Uniform:", mean_nll_uniform)
print("Overall Standard error NLL:", se_nll_uniform)

print("Overall Mean MSE dur Uniform:", mean_mse_dur_uniform)
print("Overall Standard error MSE dur Uniform:", se_mse_dur_uniform)

print("Overall Mean MSE landing pos Uniform:", mean_mse_land_pos_uniform)
print("Overall Standard error MSE landing pos:", se_mse_land_pos_uniform)

print("Overall Mean scasim Uniform:", mean_scasim_uniform)
print("Overall Standard error scasim:", se_scasim_uniform)

print("Overall Mean Central scasim Uniform:", mean_central_scasim_uniform)
print("Overall Standard error Central scasim:", se_central_scasim_uniform)


print("Overall Mean MSE dur E-Z Reader:", mean_mse_dur_ez_reader)
print("Overall Standard error MSE dur E-Z Reader:", se_mse_dur_ez_reader)

print("Overall Mean MSE landing pos E-Z Reader:", mean_mse_land_pos_ez_reader)
print("Overall Standard error MSE landing pos:", se_mse_land_pos_ez_reader)

print("Overall Mean scasim E-Z Reader:", mean_scasim_ez_reader)
print("Overall Standard error scasim:", se_scasim_ez_reader)

print("Overall Mean Central scasim E-Z Reader:", mean_central_scasim_ez_reader)
print("Overall Standard error Central scasim:", se_central_scasim_ez_reader)

Overall Mean NLL: -2.161599395801117
Overall Standard error NLL: 0.012744678584550272
Overall Mean MSE dur: 0.30829840328542085
Overall Standard error MSE dur: 0.10980872640807642
Overall Mean MSE landing pos: 0.04755040184404761
Overall Standard error MSE landing pos: 0.0014708614439799456
Overall Mean scasim: 2672.3000958772773
Overall Standard error scasim: 56.76017418510393
Overall Mean Central scasim: 1920.3509108341323
Overall Standard error Central scasim: 48.5548277385575
Human Mean scasim: 2894.0354745925215
Human Standard error scasim: 54.24253526708718
Human Mean central scasim: 1972.843720038351
Human Standard error central scasim: 48.190009637075526
Overall Mean NLL Uniform: -5.4918532371521
Overall Standard error NLL: 0.0
Overall Mean MSE dur Uniform: 5.055198182706285
Overall Standard error MSE dur Uniform: 0.37103770910942574
Overall Mean MSE landing pos Uniform: 2.153074264559334
Overall Standard error MSE landing pos: 0.03471715529452647
Overall Mean scasim Uniform: 4

In [ ]:
from scipy import stats

fold_index = 0
for i in range(5):
  with open(f'drive/MyDrive/results/celer/NRS/res_CELER_eyettention_Fold{fold_index}.pickle', 'rb') as handle:
      print(fold_index)
      fold_results = pickle.load(handle)

      # NLL
      test_ll = fold_results['test_ll'][0]

  #    with open(f'drive/MyDrive/results/celer/New_Sentence/res_CELER_original_eyettention_fold{fold_index}.pickle', 'rb') as original:
   #     original_results = pickle.load(original)
   #     original_ll = original_results['test_ll'][0]
    #    t_statistic, p_value = stats.ttest_ind(test_ll, original_ll)
        # Interpretation based on p-value
  #      if p_value < 0.05:
   #       print("Reject null hypothesis: Statistically significant difference in NLL scores between Eyettention 2.0 and original Eyettention scanpaths.")
    #    else:
     #     print("Fail to reject null hypothesis: No statistically significant difference found between Eyettention 2.0 and original Eyettention NLL scores.")

      uniform_nll = fold_results['uniform_nll'][0]

      t_statistic, p_value = stats.ttest_ind(test_ll, uniform_nll)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in NLL scores between predicted and uniformly generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between the predicted and uniform NLL scores.")

      # Scasim
      scasim_dnn = fold_results['scasim_dnn'][0]
      scasim_human = fold_results['scasim_human'][0]
      ez_reader_scasim = fold_results['ez_reader_scasim'][0]
      uniform_scasim = fold_results['uniform_scasim'][0]

      t_statistic, p_value = stats.ttest_ind(scasim_dnn, ez_reader_scasim)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in Scasim scores between predicted and E-Z Reader generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and E-Z Reader generated Scasim scores.")
      t_statistic, p_value = stats.ttest_ind(scasim_dnn, uniform_scasim)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in Scasim scores between predicted and the uniform generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and uniform Scasim scores.")

      t_statistic, p_value = stats.ttest_ind(scasim_dnn, scasim_human)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in Scasim scores between predicted and the human scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and human Scasim scores.")

      # Central Scasim
      central_scasim_dnn = fold_results['central_scasim_dnn'][0]
      central_scasim_human = fold_results['central_scasim_human'][0]
      ez_reader_central_scasim = fold_results['ez_reader_central_scasim'][0]
      uniform_central_scasim = fold_results['uniform_central_scasim'][0]

      t_statistic, p_value = stats.ttest_ind(central_scasim_dnn, ez_reader_central_scasim)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in Central Scasim scores between predicted and E-Z Reader generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and E-Z Reader Central Scasim scores.")
      t_statistic, p_value = stats.ttest_ind(central_scasim_dnn, uniform_central_scasim)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in Central Scasim scores between predicted and the uniform generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and uniform Central Scasim scores.")

      t_statistic, p_value = stats.ttest_ind(central_scasim_dnn, central_scasim_human)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in Central Scasim scores between predicted and the human scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and human Central Scasim scores.")

      # MSE durations
      test_mse_dur = fold_results['test_mse_dur'][0]
      ez_reader_mse_dur = fold_results['ez_reader_mse_dur'][0]
      uniform_mse_dur = fold_results['uniform_mse_dur'][0]

      t_statistic, p_value = stats.ttest_ind(test_mse_dur, ez_reader_mse_dur)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in MSE scores for durations between predicted and E-Z Reader generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and E-Z Reader MSE scores for durations.")
      t_statistic, p_value = stats.ttest_ind(test_mse_dur, uniform_mse_dur)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in MSE scores for durations between predicted and the uniform generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and uniform MSE scores for durations.")

      # MSE landing pos
      test_mse_land_pos = fold_results['test_mse_land_pos'][0]
      ez_reader_mse_land_pos = fold_results['ez_reader_mse_land_pos'][0]
      uniform_mse_land_pos = fold_results['uniform_mse_land_pos'][0]

      t_statistic, p_value = stats.ttest_ind(test_mse_land_pos, ez_reader_mse_land_pos)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in MSE scores for landing pos between predicted and E-Z Reader generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and E-Z Reader MSE scores for landing pos.")
      t_statistic, p_value = stats.ttest_ind(test_mse_land_pos, uniform_mse_land_pos)
      # Interpretation based on p-value
      if p_value < 0.05:
        print("Reject null hypothesis: Statistically significant difference in MSE scores for landing pos between predicted and the uniform generated scanpaths.")
      else:
        print("Fail to reject null hypothesis: No statistically significant difference between predicted and uniform MSE scores for landing pos.")

      fold_index += 1

0
Reject null hypothesis: Statistically significant difference in NLL scores between predicted and uniformly generated scanpaths.
Fail to reject null hypothesis: No statistically significant difference between predicted and E-Z Reader generated Scasim scores.
Reject null hypothesis: Statistically significant difference in Scasim scores between predicted and the uniform generated scanpaths.
Fail to reject null hypothesis: No statistically significant difference between predicted and human Scasim scores.
Reject null hypothesis: Statistically significant difference in Central Scasim scores between predicted and E-Z Reader generated scanpaths.
Reject null hypothesis: Statistically significant difference in Central Scasim scores between predicted and the uniform generated scanpaths.
Fail to reject null hypothesis: No statistically significant difference between predicted and human Central Scasim scores.
Reject null hypothesis: Statistically significant difference in MSE scores for durations

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)



Reject null hypothesis: Statistically significant difference in MSE scores for durations between predicted and the uniform generated scanpaths.
Reject null hypothesis: Statistically significant difference in MSE scores for landing pos between predicted and E-Z Reader generated scanpaths.
Reject null hypothesis: Statistically significant difference in MSE scores for landing pos between predicted and the uniform generated scanpaths.
3
Reject null hypothesis: Statistically significant difference in NLL scores between predicted and uniformly generated scanpaths.
Fail to reject null hypothesis: No statistically significant difference between predicted and E-Z Reader generated Scasim scores.
Reject null hypothesis: Statistically significant difference in Scasim scores between predicted and the uniform generated scanpaths.
Fail to reject null hypothesis: No statistically significant difference between predicted and human Scasim scores.
Fail to reject null hypothesis: No statistically signifi